# Import Libraries

In [10]:
import pandas as pd
from itertools import zip_longest
from bs4 import BeautifulSoup
from selenium import webdriver

from scipy import stats
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import time

In [11]:
# code to force show all column
pd.set_option('display.max_columns', None)

# max column width
pd.set_option('display.max_colwidth', 100)

# Load Dataset

In [12]:
df = pd.read_csv('../datasets/rumah123_hyperlink.csv')

# Scrap Web Per Pages

In [13]:
#init webdriver & test BeautifulSoup
driver = webdriver.Chrome()

url="https://www.rumah123.com/jual/rumah/?q=Balikpapan"
driver.get(url)
html = driver.page_source

soup = BeautifulSoup(html, "html.parser") #Menentukan parser HTML yang ingin kita gunakan.
print(soup.prettify()[:700]) # memberikan representasi visual dari pohon parse yang dibuat dari konten HTML mentah.

<html lang="id-ID">
 <head>
  <script async="" id="analytics-js" src="https://cdn.segment.com/analytics.js/v1/fpueXeNRJ8uBgCLuBDRDkZFAMBT0yLLb/analytics.min.js" type="text/javascript">
  </script>
  <script>
   partytown = {
                "debug": false,
                "lib": "/asset-core/script/partytown/",
                "logCalls": true,
                "logGetters": true,
                "logImageRequests": true,
                "logMainAccess": true,
                "logScriptExecution": true,
                "logSendBeaconRequests": true,
                "logStackTraces": true,
                "forward": ['dataLayer.push', 'fbq', '_hsq.push', 'ttq.track', 'ttq.page', 'ttq.load'],
 


In [14]:
# prepare variable to save the datas
id = []
judul = []
harga = []
lokasi = []
kamar_tidur = []
kamar_mandi = []
luas_tanah = []
luas_bangunan = []
carport = []
tipe_properti = []
sertifikat = []
daya_listrik = []

kamar_pembantu = []
dapur = []
ruang_makan = []
ruang_tamu = []
kondisi_perabotan = []

# Exterior
jumlah_lantai = []
pemandangan = []

# tentang properti
terjangkau_internet = []
lebar_jalan = []
sumber_air = []
hook = []
kondisi_properti = []
tipe_iklan = []
id_iklan = []

### Changed Logic
1. First initial labels of all possible data
2. Create a dictionary based on the labels
3. Create a Loop to scrap the data and store it in the dictionary
    - First, fill the dictionary with NaN for all key so that when we didn't found the data, it still in the same shape
    - Loop 1: To get the labels1 data (Kamar Tidur, Kamar Mandi, etc)
    - Loop 2: To get the labels2 data (Luas Bangunan, Luas Tanah, etc)

In [15]:
# clean dictionary
data1 = {}
data2 = {}
harga = []
judul = []
lokasi = []
harga = [np.nan]*1467
judul = [np.nan]*1467
lokasi = [np.nan]*1467
urls = [np.nan]*1467
img_hyperlinks = [np.nan]*1467

# define the labels
labels1 = ['Kamar Tidur', 'Kamar Mandi', 'Luas Tanah', 'Luas Bangunan', 'Carport', 'Tipe Properti', 'Sertifikat', 'Daya Listrik']
labels2 = ['Kamar Pembantu', 'Dapur', 'Ruang Makan', 'Ruang Tamu', 'Kondisi Perabotan', 'Jumlah Lantai', 'Pemandangan', 'Terjangkau Internet', 'Lebar Jalan', 'Sumber Air', 'Hook', 'Kondisi Properti', 'Tipe Iklan']

# create dictionaries to store the labels and their values
data1 = {label: [np.nan]*1467 for label in labels1}
data2 = {label: [np.nan]*1467 for label in labels2}

loop = 1251
print("Progresss: ")

# traverse the hyperlink to get the data needed (test 10 data first)
for i in df['hyperlink'][1251:1467]:
    
    url = i
    urls[loop] = url
    img_hyperlinks[loop] = df['img_hyperlinks'][loop]
    
    url=i
    driver.get(url)
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    
    # generate random number between 1 and 3
    rand = np.random.randint(1, 17)

    # short pause if loop is a multiple of 5, pause the execution for a specified amount of time (e.g., 60 seconds)
    if ((loop % 5) == 0) and (loop != 0):
        pause = 70 - rand
        time.sleep(pause)
        # print(f'Long Pause for {pause} seconds...')
    
    # Long pause if loop is a multiple of 19, pause the execution for a specified amount of time (e.g., 60 seconds)
    if ((loop % 19) == 0) and (loop != 0):
        pause = 90 - rand
        print(f'Long Pause for {pause} seconds...')
        time.sleep(pause)
    
    # # Super long pause if loop is a multiple of 5, pause the execution for a specified amount of time (e.g., 60 seconds)
    # if ((loop % 98) == 0) and (loop != 0):
    #     pause = 820 - rand
    #     print(f'Super Long Pause for {pause} seconds...')
    #     time.sleep(pause)
    
    
    get_price_iklan = soup.find_all('div', class_="r123-listing-summary__price")
    # get the span inside the div
    for i in get_price_iklan:
        get_price_iklan = i.find('span')  # use find instead of find_all
        harga[loop] = get_price_iklan.text

    judul_elements = soup.find_all('h1', {"class":"r123-listing-summary__header-container-title"})
    if judul_elements:
        judul[loop] = judul_elements[0].get_text()

    lokasi_elements = soup.find_all('div', {"class":"r123-listing-summary__header-container-address"})
    if lokasi_elements:
        lokasi[loop] = lokasi_elements[0].get_text()
    
    # ----------------- #

    # first loop
    spec_items = soup.find_all('div', class_="listing-specification-v2__item")
    for item in spec_items:
        label = item.find('span', class_="listing-specification-v2__item-label").text
        value = item.find('span', class_="listing-specification-v2__item-value").text

        # if the label is in the first dictionary, insert the value at the index of the current loop
        if label in data1:
            data1[label][loop] = value

    # second loop
    interior_items = soup.find_all('div', class_="ui-molecule-accordion-r123")
    for item in interior_items:
        spec_items = item.find_all('div', class_="listing-specification-v2__item")
        for spec_item in spec_items:
            label = spec_item.find('span', class_="listing-specification-v2__item-label").text
            value = spec_item.find('span', class_="listing-specification-v2__item-value").text

            # if the label is in the second dictionary, insert the value at the index of the current loop
            if label in data2:
                data2[label][loop] = value
   
    #----------------- #
    loop += 1
    
    # print loop progress total number of link divided by hyperlinks
    print(f'Done: {loop} / {df["hyperlink"].size}')    

Progresss: 
Done: 1252 / 1467
Done: 1253 / 1467
Done: 1254 / 1467
Long Pause for 74 seconds...
Done: 1255 / 1467
Done: 1256 / 1467
Done: 1257 / 1467
Done: 1258 / 1467
Done: 1259 / 1467
Done: 1260 / 1467
Done: 1261 / 1467
Done: 1262 / 1467
Done: 1263 / 1467
Done: 1264 / 1467
Done: 1265 / 1467
Done: 1266 / 1467
Done: 1267 / 1467
Done: 1268 / 1467
Done: 1269 / 1467
Done: 1270 / 1467
Done: 1271 / 1467
Done: 1272 / 1467
Done: 1273 / 1467
Long Pause for 85 seconds...
Done: 1274 / 1467
Done: 1275 / 1467
Done: 1276 / 1467
Done: 1277 / 1467
Done: 1278 / 1467
Done: 1279 / 1467
Done: 1280 / 1467
Done: 1281 / 1467
Done: 1282 / 1467
Done: 1283 / 1467
Done: 1284 / 1467
Done: 1285 / 1467
Done: 1286 / 1467
Done: 1287 / 1467
Done: 1288 / 1467
Done: 1289 / 1467
Done: 1290 / 1467
Done: 1291 / 1467
Done: 1292 / 1467
Long Pause for 80 seconds...
Done: 1293 / 1467
Done: 1294 / 1467
Done: 1295 / 1467
Done: 1296 / 1467
Done: 1297 / 1467
Done: 1298 / 1467
Done: 1299 / 1467
Done: 1300 / 1467
Done: 1301 / 1467
D

In [16]:
# convert the dictionaries to dataframes
judull = pd.DataFrame(judul, columns=['Judul'])
hargaa = pd.DataFrame(harga, columns=['Harga'])
lokasia = pd.DataFrame(lokasi, columns=['Lokasi'])
hyperlink_df = pd.DataFrame(urls, columns=['Hyperlink'])
img_hyperlink_df = pd.DataFrame(img_hyperlinks, columns=['Img_Hyperlink'])

df1 = pd.DataFrame(data1)
df2 = pd.DataFrame(data2)

# concatenate the dataframes
df_fix = pd.concat([judull, hargaa, lokasia, hyperlink_df, img_hyperlink_df, df1, df2], axis=1)
print(df_fix.shape)

# drop all rows where Judul is NaN
df_fix = df_fix.dropna(subset=['Judul'])

df_fix

(1467, 26)


,Judul,Harga,Lokasi,Hyperlink,Img_Hyperlink,Kamar Tidur,Kamar Mandi,Luas Tanah,Luas Bangunan,Carport,Tipe Properti,Sertifikat,Daya Listrik,Kamar Pembantu,Dapur,Ruang Makan,Ruang Tamu,Kondisi Perabotan,Jumlah Lantai,Pemandangan,Terjangkau Internet,Lebar Jalan,Sumber Air,Hook,Kondisi Properti,Tipe Iklan
1251,RUMAH CANTIK FULL FURNISH DI KAWASAN BELLER,"Rp 1,8 Miliar","Balikpapan Tengah, Balikpapan",https://www.rumah123.com//properti/balikpapan/hos8904125/,https://picture.rumah123.com/r123/720x420-crop/house/ho89/8904125/original/hos8904125-rumah-di-j...,5,3,150 m²,300 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,3500 Watt,NaN,NaN,Tidak,Tidak,NaN,NaN,NaN,Tidak,NaN,NaN,Tidak,NaN,Dijual
1252,RUMAH STRATEGIS DI KAWASAN PERUMAHAN KARTINI RESIDENCE,Rp 600 Juta,"Balikpapan Selatan, Balikpapan",https://www.rumah123.com//properti/balikpapan/hos8836635/,https://picture.rumah123.com/r123/720x420-crop/house/ho88/8836635/original/hos8836635-rumah-di-j...,2,1,72 m²,49 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,1300 Watt,NaN,NaN,Tidak,Tidak,NaN,NaN,NaN,Tidak,NaN,NaN,Tidak,NaN,Dijual
1253,"RUMAH MEWAH di BALIKPAPAN, KALIMANTAN TIMUR",Rp 30 Miliar,"Balikpapan Timur, Balikpapan",https://www.rumah123.com//properti/balikpapan/hos8801013/,https://picture.rumah123.com/r123/720x420-crop/house/ho88/8801013/original/hos8801013-rumah-di-j...,5,5,1400 m²,800 m²,NaN,Rumah,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dijual
1254,RUMAH BARU CANTIK KEREN SIAP HUNI DITENGAH KOTA BALIKPAPAN,Rp 600 Juta,"Balikpapan Selatan, Balikpapan",https://www.rumah123.com//properti/balikpapan/hos8796964/,https://picture.rumah123.com/r123/720x420-crop/house/ho87/8796964/original/hos8796964-rumah-di-j...,2,1,136 m²,49 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,1300 Watt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dijual
1255,RUMAH SATU LANTAI SIAP HUNI DI KAWASAN PERUMAHAN PUPUK,"Rp 2,5 Miliar","Balikpapan Selatan, Balikpapan",https://www.rumah123.com//properti/balikpapan/hos8760480/,https://picture.rumah123.com/r123/720x420-crop/house/ho87/8760480/original/hos8760480-rumah-di-j...,3,3,356 m²,182 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,6600 Watt,NaN,NaN,Tidak,Tidak,NaN,NaN,NaN,Tidak,NaN,NaN,Tidak,NaN,Dijual
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1462,Rumah Murah Harga Nego Dalam Perumahan,Rp 150 Juta,"Balikpapan Timur, Balikpapan",https://www.rumah123.com//properti/balikpapan/hos8685808/,https://picture.rumah123.com/r123/720x420-crop/house/ho86/8685808/original/hos8685808-rumah-di-j...,2,1,120 m²,36 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,900 Watt,NaN,NaN,Tidak,Tidak,Unfurnished,1,Pemukiman Warga,Ya,NaN,NaN,Tidak,Butuh Renovasi,Dijual
1463,Perumahan Elit Pupuk Kaltim,"Rp 2,7 Miliar","Balikpapan Selatan, Balikpapan",https://www.rumah123.com//properti/balikpapan/hos8219826/,https://picture.rumah123.com/r123/720x420-crop/house/ho82/8219826/original/hos8219826-rumah-di-j...,3,3,374 m²,200 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,6600 Watt,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dijual
1464,Rumah Baru Minimalis Kpr Tanpa Dp,Rp 395 Juta,"Balikpapan Utara, Balikpapan",https://www.rumah123.com//properti/balikpapan/hos8186149/,https://picture.rumah123.com/r123/720x420-crop/house/ho81/8186149/original/hos8186149-rumah-di-j...,2,1,72 m²,45 m²,1,Rumah,SHM - Sertifikat Hak Milik,1300 Watt,NaN,1,Tidak,Ya,Unfurnished,1,Pemukiman Warga,Ya,NaN,PAM atau PDAM,Ya,Baru,Dijual
1465,BALIKPAPAN POKA (Y140),"Rp 2,1 Miliar","Sepinggan, Balikpapan",https://www.rumah123.com//properti/balikpapan/hos7941758/,https://picture.rumah123.com/r123/720x420-crop/house/ho79/7941758/original/hos7941758-rumah-di-j...,5,2,403 m²,200 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,2200 Watt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dijual


In [8]:
df_fix

,Judul,Harga,Lokasi,Hyperlink,Img_Hyperlink,Kamar Tidur,Kamar Mandi,Luas Tanah,Luas Bangunan,Carport,Tipe Properti,Sertifikat,Daya Listrik,Kamar Pembantu,Dapur,Ruang Makan,Ruang Tamu,Kondisi Perabotan,Jumlah Lantai,Pemandangan,Terjangkau Internet,Lebar Jalan,Sumber Air,Hook,Kondisi Properti,Tipe Iklan
0,RUMAH 2 LANTAI TENGAH KOTA PERUMAHAN BUKIT DAMAI INDAH BALIKPAPAN KAL-TIM,"Rp 1,5 Miliar","Balikpapan Selatan, Balikpapan",https://www.rumah123.com//properti/balikpapan/hos15945789/,https://picture.rumah123.com/r123/720x420-crop/house/ho15/15945789/original/hos15945789-rumah-di...,5,3,144 m²,207 m²,1,Rumah,SHM - Sertifikat Hak Milik,4400 Watt,1,1,Tidak,Tidak,Semi Furnished,2,Pemukiman Warga,Ya,2 Mobil,PAM atau PDAM,Tidak,Bagus,Dijual
1,"Rumah Luas Lokasi Strategis di Gunung Samarinda Baru, Balikpapan Utara","Rp 3,1 Miliar","Balikpapan Utara, Balikpapan",https://www.rumah123.com//properti/balikpapan/hos16239750/,https://picture.rumah123.com/r123-images/720x420-crop/customer/1964420/2024-02-01-02-35-27-030b4...,3,2,534 m²,200 m²,3,Rumah,SHM - Sertifikat Hak Milik,2200 Watt,NaN,1,Ya,Ya,NaN,1,NaN,Ya,NaN,PAM atau PDAM,Tidak,Bagus,Dijual
2,Rumah Minimalis Baru Renovasi Untuk Keluarga Muslimin,Rp 600 Juta,"Balikpapan Utara, Balikpapan",https://www.rumah123.com//properti/balikpapan/hos15643485/,https://picture.rumah123.com/r123-images/720x420-crop/customer/1926217/2023-12-01-06-10-42-94a69...,2,2,80 m²,140 m²,1,Rumah,SHM - Sertifikat Hak Milik,1300 Watt,NaN,1,Ya,Ya,Semi Furnished,1,NaN,Ya,NaN,PAM atau PDAM,Tidak,Bagus,Dijual
3,"DIJUAL CEPAT RUMAH DI JALAN SWADAYA, KARANG REJO BALIKPAPAN",Rp 510 Juta,"Balikpapan Tengah, Balikpapan",https://www.rumah123.com//properti/balikpapan/hos14193967/,https://picture.rumah123.com/r123/720x420-crop/house/ho14/14193967/original/hos14193967-rumah-di...,3,2,201 m²,150 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,1300 Watt,NaN,NaN,Tidak,Tidak,Unfurnished,1,NaN,Tidak,NaN,NaN,Tidak,Bagus,Dijual
4,Rumah Minimalis Modern Smart Home Tanah Luas,Rp 850 Juta,"Balikpapan Utara, Balikpapan",https://www.rumah123.com//properti/balikpapan/hos15998364/,https://picture.rumah123.com/r123-images/720x420-crop/customer/1355633/2024-01-23-07-46-31-33b01...,3,3,120 m²,65 m²,1,Rumah,SHM - Sertifikat Hak Milik,2200 Watt,NaN,1,Ya,Ya,Unfurnished,2,Pemukiman Warga,Ya,NaN,Sumur Bor,Tidak,Baru,Dijual
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,RUMAH BALIKPAPAN WIKA HOOK BARU RENOV,"Rp 1,69 Miliar","Balikpapan Utara, Balikpapan",https://www.rumah123.com//properti/balikpapan/hos9359364/,https://picture.rumah123.com/r123/720x420-crop/house/ho93/9359364/original/hos9359364-rumah-di-j...,3,1,250 m²,60 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,1300 Watt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dijual
362,BALIKPAPAN GRANDCITY (Y199),"Rp 1,1 Miliar","Balikpapan Selatan, Balikpapan",https://www.rumah123.com//properti/balikpapan/hos8247708/,https://picture.rumah123.com/r123/720x420-crop/house/ho82/8247708/original/hos8247708-rumah-di-j...,2,2,72 m²,57 m²,NaN,Rumah,"Lainnya (PPJB,Girik,Adat,dll)",1300 Watt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dijual
363,"BALIKPAPAN, WIKA RUMAH ASRI TERAWAT (Y288)","Rp 1,2 Miliar","Balikpapan Utara, Balikpapan",https://www.rumah123.com//properti/balikpapan/hos8834933/,https://picture.rumah123.com/r123/720x420-crop/house/ho88/8834933/original/hos8834933-rumah-di-j...,3,2,180 m²,120 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,4400 Watt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dijual
364,RUMAH HOMY MEWAH LUAS SIAP HUNI BALIKPAPAN,"Rp 4,6 Miliar","Balikpapan Selatan, Balikpapan",https://www.rumah123.com//properti/balikpapan/hos8787857/,https://picture.rumah123.com/r123/720x420-crop/house/ho87/8787857/original/hos8787857-rumah-di-j...,3,2,485 m²,300 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,6600 Watt,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dijual


In [17]:
# to csv
df_fix.to_csv('rumah123_data_single_raw_fix_part_3_b_complete.csv', index=True)

In [9]:
# full inner join on judul and Judul delete the rest that not match
df_fix_combine = pd.merge(df, df_fix, left_on='judul', right_on='Judul', how='inner')

# drop where Judul is NaN
df_fix_combine = df_fix_combine.dropna(subset=['Judul'])

In [10]:
df_fix_combine.sample(10)

,judul,hyperlink,img_hyperlinks,Judul,Harga,Lokasi,Hyperlink,Img_Hyperlink,Kamar Tidur,Kamar Mandi,Luas Tanah,Luas Bangunan,Carport,Tipe Properti,Sertifikat,Daya Listrik,Kamar Pembantu,Dapur,Ruang Makan,Ruang Tamu,Kondisi Perabotan,Jumlah Lantai,Pemandangan,Terjangkau Internet,Lebar Jalan,Sumber Air,Hook,Kondisi Properti,Tipe Iklan
134,DIJUAL RUMAH PINGGIR JALAN RAYA UTAMA GUNUNG MALANG,https://www.rumah123.com//properti/balikpapan/hos13121565/,https://picture.rumah123.com/r123/720x420-crop/house/ho13/13121565/original/hos13121565-rumah-di...,DIJUAL RUMAH PINGGIR JALAN RAYA UTAMA GUNUNG MALANG,Rp 7 Miliar,"Klandasan Ilir, Balikpapan",https://www.rumah123.com//properti/balikpapan/hos13121565/,https://picture.rumah123.com/r123/720x420-crop/house/ho13/13121565/original/hos13121565-rumah-di...,3,1,320 m²,270 m²,4,Rumah,SHM - Sertifikat Hak Milik,2200 Watt,NaN,1,Tidak,Tidak,Unfurnished,2,Perkotaan,Tidak,4 Mobil,PAM atau PDAM,Tidak,Butuh Renovasi,Dijual
24,RUMAH MEWAH RASA VILA DI PERUMAHAN PUPUK,https://www.rumah123.com//properti/balikpapan/hos16245587/,https://picture.rumah123.com/r123/720x420-crop/house/ho16/16245587/original/hos16245587-rumah-di...,RUMAH MEWAH RASA VILA DI PERUMAHAN PUPUK,"Rp 5,25 Miliar","Balikpapan Selatan, Balikpapan",https://www.rumah123.com//properti/balikpapan/hos16245587/,https://picture.rumah123.com/r123/720x420-crop/house/ho16/16245587/original/hos16245587-rumah-di...,3,3,500 m²,350 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,16500 Watt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dijual
64,RUMAH TEPI JALAN BATU AMPAR,https://www.rumah123.com//properti/balikpapan/hos15487505/,https://picture.rumah123.com/r123/720x420-crop/house/ho15/15487505/original/hos15487505-rumah-di...,RUMAH TEPI JALAN BATU AMPAR,"Rp 1,9 Miliar","Balikpapan Utara, Balikpapan",https://www.rumah123.com//properti/balikpapan/hos15487505/,https://picture.rumah123.com/r123/720x420-crop/house/ho15/15487505/original/hos15487505-rumah-di...,4,4,276 m²,250 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,3500 Watt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dijual
44,RUMAH DI DAUN VILLAGE,https://www.rumah123.com//properti/balikpapan/hos15842287/,https://picture.rumah123.com/r123/720x420-crop/house/ho15/15842287/original/hos15842287-rumah-di...,RUMAH DI DAUN VILLAGE,"Rp 1,2 Miliar","Balikpapan Selatan, Balikpapan",https://www.rumah123.com//properti/balikpapan/hos15842287/,https://picture.rumah123.com/r123/720x420-crop/house/ho15/15842287/original/hos15842287-rumah-di...,3,1,136 m²,85 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,1300 Watt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dijual
116,"RUMAH SIAP HUNI, DALAM KOTA,BEBAS BANJIR",https://www.rumah123.com//properti/balikpapan/hos14159226/,https://picture.rumah123.com/r123/720x420-crop/house/ho14/14159226/original/hos14159226-rumah-di...,"RUMAH SIAP HUNI, DALAM KOTA,BEBAS BANJIR",Rp 530 Juta,"Balikpapan Utara, Balikpapan",https://www.rumah123.com//properti/balikpapan/hos14159226/,https://picture.rumah123.com/r123/720x420-crop/house/ho14/14159226/original/hos14159226-rumah-di...,2,2,100 m²,90 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,1300 Watt,NaN,NaN,Tidak,Tidak,NaN,NaN,NaN,Tidak,NaN,NaN,Tidak,NaN,Dijual
141,BALIKPAPAN RUMAH HOOK MEWAH PERUMAHAN BALIKPAPAN BARU (HM136),https://www.rumah123.com//properti/balikpapan/hos13117250/,https://picture.rumah123.com/r123/720x420-crop/house/ho13/13117250/original/hos13117250-rumah-di...,BALIKPAPAN RUMAH HOOK MEWAH PERUMAHAN BALIKPAPAN BARU (HM136),"Rp 7,5 Miliar","Balikpapan Baru, Balikpapan",https://www.rumah123.com//properti/balikpapan/hos13117250/,https://picture.rumah123.com/r123/720x420-crop/house/ho13/13117250/original/hos13117250-rumah-di...,6,4,575 m²,900 m²,NaN,Rumah,SHM - Sertifikat Hak Milik,13300 Watt,NaN,NaN,Tidak,Tidak,NaN,NaN,NaN,Tidak,NaN,NaN,Tidak,NaN,Dijual
136,DIJUAL RUMAH MODERN 2 MENIT DARI STT MIGAS KM 8 BALIKPAPAN,https://www.rumah123.com//properti/balikpapan/hos13120796/,https://picture.rumah123.com/r123/720x420-crop/house/ho13/13120796

In [11]:
# check null values
df_fix_combine.isnull().sum()

judul                    0
hyperlink                0
img_hyperlinks           0
Judul                    0
Harga                    0
Lokasi                   0
Hyperlink                0
Img_Hyperlink            0
Kamar Tidur              8
Kamar Mandi              9
Luas Tanah               0
Luas Bangunan            0
Carport                129
Tipe Properti            0
Sertifikat               0
Daya Listrik            10
Kamar Pembantu         125
Dapur                  119
Ruang Makan             72
Ruang Tamu              72
Kondisi Perabotan      102
Jumlah Lantai          100
Pemandangan            112
Terjangkau Internet     72
Lebar Jalan            113
Sumber Air             117
Hook                    72
Kondisi Properti        91
Tipe Iklan               0
dtype: int64